In [6]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from dt_apriltags import Detector
from pid import PID


In [9]:
def captureFrame():
    cap = cv2.VideoCapture('AprilTagTest.mkv')
    success = cap.grab()
    frames = []
    at_detector = Detector(families='tag36h11',
                        nthreads=1,
                        quad_decimate=1.0,
                        quad_sigma=0.0,
                        refine_edges=1,
                        decode_sharpening=0.25,
                        debug=0)
    cameraMatrix = np.array([ 1060.71, 0, 960, 0, 1060.71, 540, 0, 0, 1]).reshape((3,3))
    camera_params = (cameraMatrix[0,0], cameraMatrix[1,1], cameraMatrix[0,2], cameraMatrix[1,2] )
    i=0
    # tags[0].pose_t
    # tags[0].pose_R
    centers = []
    while success:
        i+=1
        if i % 100  == 0:
            _, frame = cap.retrieve()
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            tags = at_detector.detect(frame, True, camera_params, 0.1)
            color_img = cv2.cvtColor(frame, cv2.COLOR_GRAY2RGB)
            for tag in tags:
                for idx in range(len(tag.corners)):
                    cv2.line(color_img, tuple(tag.corners[idx - 1, :].astype(int)), tuple(tag.corners[idx, :].astype(int)), (0, 255, 0))
                    cv2.putText(color_img, str(tag.tag_id),
                    org=(tag.corners[0, 0].astype(int) + 10, tag.corners[0, 1].astype(int) + 10),
                    fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale=0.8,
                    color=(0, 0, 255))
                (x, y) = (int(tag.center[0]), int(tag.center[1]))
                centers.append((x,y))
                cv2.circle(frame, (x, y), 5, (225, 0, 0), -1) # not drawing a circle :(
            frames.append(color_img)
            #plt.imshow(color_img)
            #plt.pause(0.01)  # Pause for a short time to show the figure
            #plt.clf()
        success = cap.grab()
    cap.release()
    #plt.close()
    return frames, centers

In [10]:
def getDistance(frames, centers):  
    height, width, layers = frames[0].shape
    distances = np.zeros((len(centers), 2))
    y_center = width/2
    x_center = height/2
    for i in range(len(centers)):
        Xa, Ya = centers[i]
        Cx = x_center-Xa
        Cy = y_center-Ya
        distances[i][0] = Cx
        distances[i][1] = Cy
    return distances

In [17]:
pidX = PID(30, 0, 0, 100)
pidY = PID(30, 0, 0, 100)
frames, centers = captureFrame()
print("PID (X,Y)")
distances = getDistance(frames, centers)
for i in range(len(distances)):
    output1 = pidX.update(distances[i][0])
    output2 = pidY.update(distances[i][1])
    print(output1, output2)


PID (X,Y)
-17940.0 16110.0
-8880.0 21750.0
-20550.0 2130.0
-1710.0 13050.0
-19020.0 17730.0
